<a href="https://colab.research.google.com/github/PriyankaMittapelly/SaiPriyanka_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

(1) Features (top n-gram phrases) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 


In [40]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

df = pd.read_csv('/content/datacleaning11.csv')

df.head(101)

# Convert to list
data = df['DATAafter_cleaning'].tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#print(data)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words)

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words]])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
clean_text = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(clean_text)

# Create Dictionary
id2word = corpora.Dictionary(clean_text)

# Create Corpus
texts = clean_text

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
print(lda_model.print_topics())

[(0, '0.045*"current" + 0.035*"focu" + 0.035*"chapter" + 0.024*"nation" + 0.024*"year" + 0.024*"work" + 0.024*"develop" + 0.024*"last" + 0.024*"computerassist" + 0.024*"thirtyf"'), (1, '0.046*"find" + 0.035*"word" + 0.029*"thesauru" + 0.018*"differ" + 0.018*"year" + 0.018*"gener" + 0.018*"keyfact" + 0.012*"need" + 0.012*"syntact" + 0.012*"support"'), (2, '0.028*"human" + 0.028*"approach" + 0.014*"class" + 0.014*"automat" + 0.014*"way" + 0.014*"complex" + 0.014*"agent" + 0.014*"interpret" + 0.014*"result" + 0.014*"speak"'), (3, '0.023*"effect" + 0.022*"test" + 0.017*"linguist" + 0.017*"state" + 0.017*"type" + 0.012*"recent" + 0.012*"present" + 0.012*"also" + 0.012*"applic" + 0.012*"result"'), (4, '0.024*"target" + 0.024*"tutor" + 0.024*"limit" + 0.012*"word" + 0.012*"current" + 0.012*"automat" + 0.012*"object" + 0.012*"document" + 0.012*"method" + 0.012*"gener"'), (5, '0.033*"logic" + 0.026*"program" + 0.026*"word" + 0.020*"linguist" + 0.020*"process" + 0.020*"give" + 0.020*"document" +

In [38]:
#LSA MODEL

from gensim.models import LdaModel
import gensim.corpora as corpora

def gensim_lsa_model(dictionary,corpus,number_of_topics,words):
    lsamodel = LsiModel(corpus, num_topics=number_of_topics, id2word = dictionary)  # train model
    pprint(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel
def coherence_values_lsa(dictionary, corpus, doc_clean, stop, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(corpus, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    print(coherence_values)
    return model_list, coherence_values
# LSA Model
from gensim.models import LsiModel

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
print(lda_model.print_topics())

[(0, '0.045*"current" + 0.035*"focu" + 0.035*"chapter" + 0.024*"nation" + 0.024*"year" + 0.024*"work" + 0.024*"develop" + 0.024*"last" + 0.024*"computerassist" + 0.024*"thirtyf"'), (1, '0.046*"find" + 0.035*"word" + 0.029*"thesauru" + 0.018*"differ" + 0.018*"year" + 0.018*"gener" + 0.018*"keyfact" + 0.012*"need" + 0.012*"syntact" + 0.012*"support"'), (2, '0.028*"human" + 0.028*"approach" + 0.014*"class" + 0.014*"automat" + 0.014*"way" + 0.014*"complex" + 0.014*"agent" + 0.014*"interpret" + 0.014*"result" + 0.014*"speak"'), (3, '0.023*"effect" + 0.022*"test" + 0.017*"linguist" + 0.017*"state" + 0.017*"type" + 0.012*"recent" + 0.012*"present" + 0.012*"also" + 0.012*"applic" + 0.012*"result"'), (4, '0.024*"target" + 0.024*"tutor" + 0.024*"limit" + 0.012*"word" + 0.012*"current" + 0.012*"automat" + 0.012*"object" + 0.012*"document" + 0.012*"method" + 0.012*"gener"'), (5, '0.033*"logic" + 0.026*"program" + 0.026*"word" + 0.020*"linguist" + 0.020*"process" + 0.020*"give" + 0.020*"document" +

In [ ]:
Summarize and describe the topic for each cluster.

0, '0.045*"current" + 0.035*"focu" + 0.035*"chapter" + 0.024*"nation" + 0.024*"year" + 0.024*"work" + 0.024*"develop" + 0.024*"last" + 0.024*"computerassist" + 0.024*"thirtyf"'),
 (1, '0.046*"find" + 0.035*"word" + 0.029*"thesauru" + 0.018*"differ" + 0.018*"year" + 0.018*"gener" + 0.018*"keyfact" + 0.012*"need" + 0.012*"syntact" + 0.012*"support"'),
 (2, '0.028*"human" + 0.028*"approach" + 0.014*"class" + 0.014*"automat" + 0.014*"way" + 0.014*"complex" + 0.014*"agent" + 0.014*"interpret" + 0.014*"result" + 0.014*"speak"'),
 (3, '0.023*"effect" + 0.022*"test" + 0.017*"linguist" + 0.017*"state" + 0.017*"type" + 0.012*"recent" + 0.012*"present" + 0.012*"also" + 0.012*"applic" + 0.012*"result"'),
 (4, '0.024*"target" + 0.024*"tutor" + 0.024*"limit" + 0.012*"word" + 0.012*"current" + 0.012*"automat" + 0.012*"object" + 0.012*"document" + 0.012*"method" + 0.012*"gener"'), 
(5, '0.033*"logic" + 0.026*"program" + 0.026*"word" + 0.020*"linguist" + 0.020*"process" + 0.020*"give" + 0.020*"document" + 0.013*"model" + 0.013*"class" + 0.013*"help"'), 
(6, '0.040*"model" + 0.020*"recent" + 0.020*"particular" + 0.020*"develop" + 0.020*"present" + 0.020*"indian" + 0.011*"make" + 0.011*"form" + 0.011*"work" + 0.011*"level"'), 
(7, '0.030*"pattern" + 0.022*"work" + 0.018*"process" + 0.018*"music" + 0.013*"statist" + 0.013*"extract" + 0.013*"confid" + 0.013*"limit" + 0.013*"develop" + 0.013*"understand"'),
 (8, '0.025*"develop" + 0.017*"lyric" + 0.017*"music" + 0.017*"way" + 0.017*"concern" + 0.017*"statist" + 0.017*"includ" + 0.017*"review" + 0.017*"deep" + 0.017*"recent"'),
 (9, '0.032*"visual" + 0.032*"automat" + 0.021*"develop" + 0.021*"model" + 0.021*"graph" + 0.021*"datum" + 0.021*"entri" + 0.021*"maximum" + 0.011*"understand" + 0.011*"perform"')
Summarizing the topic names based on my knowledge with the help of keywords extracted using LDA and LSA model . 
As that models extracts the keywords that define the sentence by removing unwanted information to get an idea on the topic 
topic 0 : programming , it has keywords like computerassist , develop , work
topic 1 :  support , its may be categorized into supporting something . we can estimate by looking at words find , need , support but not sure 
topic 3: test results  , its may be categraized into that topic because it has keywords application , results ,test 
topic 4: goal , its about reaching some target can be identified by words target , limit , object 
topic 5 : assignmnet , it consistys of keywords that are related to homework , class, help , work , program
topic 6:country , it talks about a country(india ) work , development 
topic 7 : song composition , it consists of keywords work , pattern , develop , music
topic 8: Music  , it consists keywords like lyric , music , deep , review
topic 9 : development , because its consists of keywords like develop , understand , graph

# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. 

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9. 

In [ ]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Cleaning the data , however  data was cleaned in previous assignment 3 

df1 = pd.read_csv('/content/datacleaning11.csv')
df1['Sentimental'] = df1['Sentimental'].replace(['positive'] ,1 )
df1['Sentimental'] = df1['Sentimental'].replace(['neutral'] ,1 )
df1['Sentimental'] = df1['Sentimental'].replace(['negative'] ,0)
df = pd.DataFrame(df1)

data1 = df


stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=True , lowercase = True  , stop_words=stopset)

df


,DATAafter_cleaning,Sentimental
0,found,1
1,describ method statist model base maximum entr...,1
2,seal condit random field term condit term cond...,1
3,address issu cooper linguist gener linguist tr...,1
4,descript logic use incom base syntact semant t...,0
...,...,...
95,present workbench built priberam informática d...,1
96,abstract—natur effect bring improv educ set im...,1
97,twenti year disfavor technolog return imit pro...,1
98,statist frequent use stylometri cryptographi s...,1


In [ ]:
y = df.Sentimental
x = vectorizer.fit_transform(df.DATAafter_cleaning)
print(y.shape)
print(x.shape)
X_train, X_test, y_train, y_test = train_test_split(x , y  ,random_state=43)
clf = naive_bayes.MultinomialNB()
clf.fit(X_train , y_train)


accuracy = roc_auc_score(y_test , clf.predict_proba(X_test)[:,-1])

print("Accuracy {}" .format(accuracy))


(100,)
(100, 816)
Accuracy 0.625


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from  sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
#print(data1)
vectorizer = CountVectorizer(stop_words='english')
all_features = vectorizer.fit_transform(data1.DATAafter_cleaning)
all_features.shape
vectorizer.vocabulary_
X_train, X_test, y_train, y_test = train_test_split(all_features , data1.Sentimental , test_size = 0.2 , random_state = 88)
X_train.shape
X_test.shape
classifier = MultinomialNB()
classifier.fit(X_train , y_train)
nr_correct = (y_test == classifier.predict(X_test)).sum()
classifier.predict(X_test)
Accuracy =  classifier.score(X_test , y_test)
print(" Accuracy {}" .format(Accuracy))


 Accuracy 0.65


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
mnb = MultinomialNB()
svm = LinearSVC()
xgb = XGBClassifier()
scores = cross_val_score(mnb, X_test, y_test, cv=10)
print("using MNB",scores.mean())
model_svm = svm.fit(X_train,y_train)
y_pred_svm = model_svm.predict(X_test)
print('Accuracy %s' % accuracy_score(y_pred_svm,y_test))
print(classification_report(y_test,y_pred_svm))
mnb = MultinomialNB()
tfid = TfidfVectorizer()
svm = LinearSVC()
rf = RandomForestClassifier()
scores = cross_val_score(rf, X_test, y_test, cv=10)
#print("using random forest",scores.mean())
model_xgb = xgb.fit(X_train,y_train)
y_pred_xgb = model_xgb.predict(X_test)
print('Accuracy %s' % accuracy_score(y_pred_xgb,y_test))
print(classification_report(y_test,y_pred_xgb))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


using MNB 0.9666666666666666
Accuracy 0.96
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.96      1.00      0.98        24

    accuracy                           0.96        25
   macro avg       0.48      0.50      0.49        25
weighted avg       0.92      0.96      0.94        25

Accuracy 0.96
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.96      1.00      0.98        24

    accuracy                           0.96        25
   macro avg       0.48      0.50      0.49        25
weighted avg       0.92      0.96      0.94        25



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download here: https://github.com/unt-iialab/info5731_spring2021/blob/main/assignment/assignment4-question3-data.zip. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

df_train.info()
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [43]:
df_test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,1458.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1457.000000,1457.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.00000,1381.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,554.294925,1046.117970,1156.534613,325.967786,3.543523,1486.045922,0.434454,0.065202,1.570939,0.377656,2.854010,1.042495,6.385195,0.58122,1977.721217,1.766118,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,176.753926,437.260486,442.898624,398.165820,420.610226,44.043251,485.566099,0.530648,0.252468,0.555190,0.503017,0.829788,0.208472,1.508895,0.64742,26.431175,0.775945,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,407.000000,0.000000,0.000000,407.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.00000,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,219.250000,784.000000,873.500000,0.000000,0.000000,1117.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.00000,1959.000000,1.000000,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,0.000000,460.000000,988.000000,1079.000000,0.000000,0.000000,1432.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,0.00000,1979.000000,2.000000,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,0.000000,797.750000,1305.000000,1382.500000,676.000000,0.000000,1721.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.00000,2002.000000,2.000000,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,2140.000000,5095.000000,5095.000000,1862.000000,1064.000000,5095.000000,3.000000,2.000000,4.000000,2.000000,6.000000,2.000000,15.000000,4.00000,2207.000000,5.000000,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [44]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [48]:
corr_matrix = df_train.corr()
corr_matrix

corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [49]:
non = pd.DataFrame(df_train.isnull().sum().sort_values(ascending=False)[:20])
non.columns = ['Null Count']
non

,Null Count
PoolQC,1453
MiscFeature,1406
Alley,1369
Fence,1179
FireplaceQu,690
LotFrontage,259
GarageCond,81
GarageType,81
GarageYrBlt,81
GarageFinish,81


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
df_train = df_train.select_dtypes(include=[np.number]).interpolate().dropna()
df_test = df_test.select_dtypes(include=[np.number]).interpolate().dropna()
a1 = df_train.drop(['SalePrice','Id'], axis=1)
a2 = np.log(df_train.SalePrice)
x_train, x_test, y_train, y_test = train_test_split(a1,a2, random_state= 42, test_size=0.2)
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
y_LR = linear_model.predict(x_test)
linear_model.intercept_,linear_model.coef_

(13.988866348992978,
 array([-7.65018361e-04, -5.95665261e-04,  1.87661534e-06,  8.57816051e-02,
         4.42889038e-02,  2.93084751e-03,  1.31197093e-03, -2.09282654e-05,
         2.17519038e-05,  6.09968394e-06,  4.57918851e-06,  3.24307762e-05,
         4.47322844e-05,  1.88316389e-05,  8.76394091e-05,  1.51203332e-04,
         7.24527026e-02,  1.55431436e-02,  3.24099866e-02,  2.21788209e-02,
         1.96120990e-03, -4.47403482e-02,  1.43675969e-02,  4.95230875e-02,
        -3.29262408e-04,  7.44644907e-02,  3.83372117e-05,  1.20796329e-04,
        -4.00325519e-05,  1.50880502e-04,  2.94041692e-04,  3.29046241e-04,
        -4.00360397e-04, -2.49339566e-06,  3.09082461e-04, -5.51616325e-03]))

In [62]:
print('Linear Regression:', linear_model.score(x_test,y_test))
MSE = np.square(np.subtract(y_test,y_LR)).mean()
print("Mean Squared Error:", MSE)





Linear Regression: 0.8758658336149933
Mean Squared Error: 0.023165113523875666
